# Load your data

Before finetuning a pretrained model of the experiments we provide in our repository (or precomputed and provided [here](https://datacloud.hhi.fraunhofer.de/nextcloud/s/NCjYws3mamLrkKq)), first load your custom 100 Hz sampled 12-lead ECG signal data `X` of shape `[N,L,12]` in Millivolts (mV) and multi-hot encoded labels `y` of shape `[N,C]` as numpy arrays, where `C` is the number of classes and `N` the number of total samples in this dataset. Although PTB-XL comes with fixed `L=1000` (i,e. 10 seconds), it is not required to be fixed, **BUT** the shortest sample must be longer than `input_size` of the specific model (e.g. 2.5 seconds for our fastai-models).

For proper tinetuning split your data into four numpy arrays: `X_train`,`y_train`,`X_val` and `y_val`

### Example: finetune model trained on all (71) on superdiagnostic (5)
Below we provide an example for loading [PTB-XL](https://physionet.org/content/ptb-xl/1.0.1/) aggregated at the `superdiagnostic` level, where we use the provided folds for train-validation-split:

In [1]:
from ecg_ptbxl_benchmarking.code.utils import utils

sampling_frequency=100
datafolder='/global/D1/homes/jayao/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.2/ptbxl/'
task='superdiagnostic'
outputfolder='/global/D1/homes/jayao/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.2/output/'

# Load PTB-XL data
data, raw_labels = utils.load_dataset(datafolder, sampling_frequency)
# Preprocess label data
labels = utils.compute_label_aggregations(raw_labels, datafolder, task)
# Select relevant data and convert to one-hot
data, labels, Y, _ = utils.select_data(data, labels, task, min_samples=0, outputfolder=outputfolder)

# 1-9 for training 
X_train = data[labels.strat_fold < 10]
y_train = Y[labels.strat_fold < 10]
# 10 for validation
X_val = data[labels.strat_fold == 10]
y_val = Y[labels.strat_fold == 10]

num_classes = 5         # <=== number of classes in the finetuning dataset
input_shape = [1000,12] # <=== shape of samples, [None, 12] in case of different lengths

X_train.shape, y_train.shape, X_val.shape, y_val.shape

((19225, 1000, 12), (19225, 5), (2158, 1000, 12), (2158, 5))

# Train or download models
There are two possibilities:
   1. Run the experiments as described in README. Afterwards you find trained in models in `output/expX/models/`
   2. Download the precomputed `output`-folder with all experiments and models from [here]((https://datacloud.hhi.fraunhofer.de/nextcloud/s/NCjYws3mamLrkKq))

# Load pretrained model

For loading a pretrained model:
   1. specify `modelname` which can be seen in `code/configs/` (e.g. `modelname='fastai_xresnet1d101'`)
   2. provide `experiment` to build the path `pretrainedfolder` (here: `exp0` refers to the experiment with `all` 71 SCP-statements)
   
This returns the pretrained model where the classification is replaced by a random initialized head with the same number of outputs as the number of classes.

In [7]:
from ecg_ptbxl_benchmarking.code.models.fastai_model import fastai_model

experiment = 'test'
modelname = 'fastai_xresnet1d101'
pretrainedfolder = '/global/D1/homes/jayao/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.2/output/'+experiment+'/models/'+modelname+'/'
mpath='/global/D1/homes/jayao/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.2/output/' # <=== path where the finetuned model will be stored
n_classes_pretrained = 5 # <=== because we load the model from exp0, this should be fixed because this depends the experiment

model = fastai_model(
    modelname, 
    num_classes, 
    sampling_frequency, 
    mpath, 
    input_shape=input_shape, 
    pretrainedfolder=pretrainedfolder,
    n_classes_pretrained=n_classes_pretrained, 
    pretrained=True,
    epochs_finetuning=2,
)

target_fs: 100
input_size: 100
input_channels: 12
chunkify_train: False
chunkify_valid: True
min_chunk_length: 100


# Preprocess data with pretrained Standardizer

Since we standardize inputs to zero mean and unit variance, your custom data needs to be standardized with the respective mean and variance. This is also provided in the respective experiment folder `output/expX/data/standard_scaler.pkl`

In [8]:
import pickle

# standard_scaler = pickle.load(open('../output/'+experiment+'/data/standard_scaler.pkl', "rb"))
standard_scaler = pickle.load(open('/global/D1/homes/jayao/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.2/output/'+experiment+'/data/standard_scaler.pkl', "rb"))

X_train = utils.apply_standardizer(X_train, standard_scaler)
X_val = utils.apply_standardizer(X_val, standard_scaler)

# Finetune model

Calling `model.fit` of a model with `pretrained=True` will perform finetuning as proposed in our work i.e. **gradual unfreezing and discriminative learning rates**. 

In [9]:
model.fit(X_train, y_train, X_val, y_val)

Finetuning...
(19225, 2)
(2158, 2)
model: fastai_xresnet1d101


epoch,train_loss,valid_loss,accuracy_multi,balanced_accuracy_multi,precision_multi,recall_multi,specificity_multi,F1_multi,time


LR Finder is complete, type {learner_name}.recorder.plot() to see the graph.


epoch,train_loss,valid_loss,accuracy_multi,balanced_accuracy_multi,precision_multi,recall_multi,specificity_multi,F1_multi,time
0,0.287521,0.309373,0.874845,0.815511,0.798704,0.690007,0.941015,0.736708,00:17
1,0.255112,0.293629,0.879752,0.827758,0.795500,0.718602,0.936915,0.751746,00:17


epoch,train_loss,valid_loss,accuracy_multi,balanced_accuracy_multi,precision_multi,recall_multi,specificity_multi,F1_multi,time


LR Finder is complete, type {learner_name}.recorder.plot() to see the graph.


epoch,train_loss,valid_loss,accuracy_multi,balanced_accuracy_multi,precision_multi,recall_multi,specificity_multi,F1_multi,time
0,0.248278,0.296196,0.879591,0.828834,0.792657,0.722174,0.935495,0.752364,00:18
1,0.248823,0.297877,0.879567,0.827784,0.794571,0.718901,0.936669,0.751475,00:19


# Evaluate model on validation data

In [10]:
y_val_pred = model.predict(X_val)
utils.evaluate_experiment(y_val, y_val_pred)

(2158, 2)
(2158, 2)
model: fastai_xresnet1d101


aggregating predictions...


,macro_auc
0,0.920827


### Finetune custom model

In [11]:
from ecg_ptbxl_benchmarking.code.utils import utils

sampling_frequency=100
datafolder='/global/D1/homes/jayao/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.2/ptbxl/'
task='superdiagnostic'
outputfolder='/global/D1/homes/jayao/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.2/output/'

# Load PTB-XL data
data, raw_labels = utils.load_dataset(datafolder, sampling_frequency)
# Preprocess label data
labels = utils.compute_label_aggregations(raw_labels, datafolder, task)
# Select relevant data and convert to one-hot
data, labels, Y, _ = utils.select_data(data, labels, task, min_samples=0, outputfolder=outputfolder)

# 1-9 for training 
X_train = data[labels.strat_fold < 10]
y_train = Y[labels.strat_fold < 10]
# 10 for validation
X_val = data[labels.strat_fold == 10]
y_val = Y[labels.strat_fold == 10]

num_classes = 5         # <=== number of classes in the finetuning dataset
input_shape = [1000,12] # <=== shape of samples, [None, 12] in case of different lengths

X_train.shape, y_train.shape, X_val.shape, y_val.shape

((19225, 1000, 12), (19225, 5), (2158, 1000, 12), (2158, 5))

In [ ]:
import torch
import torch.nn as nn
import numpy as np

#for losses_plot
import matplotlib
import matplotlib.pyplot as plt

import torch.optim as optim
from torch.optim.lr_scheduler import CyclicLR, ReduceLROnPlateau
from torch.utils.data import DataLoader, TensorDataset


pretrainedfolder = '/global/D1/homes/jayao/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.2/output/models/'
# your_model.pth
mpath='/global/D1/homes/jayao/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.2/output/custom_test/models/' # <=== path where the finetuned model will be stored
n_classes_pretrained = 5 # <=== because we load the model from exp0, this should be fixed because this depends the experiment

# Define the Residual Block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        
        # First convolutional layer
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm1d(out_channels)
        
        # ReLU activation function
        self.relu = nn.ReLU(inplace=True)
        
        # Second convolutional layer
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm1d(out_channels)
        
        # Shortcut connection (if needed)
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv1d(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm1d(out_channels)
            )
        else:
            self.shortcut = nn.Identity()
    
    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        
        out = self.conv2(out)
        out = self.bn2(out)
        
        shortcut = self.shortcut(x)
        
        out += shortcut  # Element-wise addition
        out = self.relu(out)
        return out

# Define the ResNet-like model
class YourModel(nn.Module):
    def __init__(self, categories):
        super(YourModel, self).__init__()
        # Define the layers for the model
        self.conv1 = nn.Conv1d(12, 32, kernel_size=5, stride=1, padding=2)  # Adjust the input channels to match the input data
        self.block1 = ResidualBlock(32, 32)
        self.block2 = ResidualBlock(32, 32)
        self.fc1 = nn.Linear(32000, 32)  # Adjust the input size based on your data
        self.fc2 = nn.Linear(32, categories)  # Updated to match the desired number of output categories
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.block1(x)
        x = self.block2(x)
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        x = torch.sigmoid(x)  # Apply sigmoid activation
        return x
    
    def accuracy_multi(self, inp, targ, thresh=0.5, sigmoid=True):
        if sigmoid:
            inp = inp.sigmoid()
        return ((inp > thresh) == targ.bool()).float().mean()
    
    def save_model(self, path):
        torch.save(self.state_dict(), path)

    @classmethod
    def load_model(cls, path, categories=5):
        model = cls(categories=categories)  # Create an instance of your model class
        pretrained_file = os.path.join(path, 'your_model.pth')  # Specify the file name of the pre-trained model
        model.load_state_dict(torch.load(pretrained_file))
        model.eval()  # Set the model to evaluation mode
        return model

        
    def fit(self, X_train, y_train, X_val, y_val, sigmoid= True, model_save_path=None):
        # Convert data to the appropriate data type (e.g., torch.float32)
        X_train = torch.FloatTensor(X_train)
        y_train = torch.FloatTensor(y_train)
        X_val = torch.FloatTensor(X_val)
        y_val = torch.FloatTensor(y_val)

        X_train = X_train.permute(0, 2, 1)  # Permute the dimensions to change to [batch_size, 12, 1000]
        X_val = X_val.permute(0, 2, 1)  # Permute the dimensions to change to [batch_size, 12, 1000]

        # Implement the training process for your model here
        # You should use torch.nn.Module's optimization, loss, and train loop
        # Make sure your model is compatible with float input and target data

        # Example of training loop (update with your model and data):
        optimizer = optim.Adam(self.parameters(), lr=0.001)
        # criterion = nn.BCELoss() 
        criterion = nn.BCEWithLogitsLoss() 

        epochs = 100
        batch_size = 128

        scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)


        train_losses = []  # To store training losses
        val_losses = []    # To store validation losses
        train_accuracies = []
        val_accuracies = []

        for epoch in range(epochs):
            self.train()
            num_batches = len(X_train) // batch_size

            num_correct = 0
            num_samples = 0
            train_loss = 0.0
            val_loss = 0.0

            for batch in range(num_batches):
                start = batch * batch_size
                end = (batch + 1) * batch_size
                batch_X = X_train[start:end]
                batch_y = y_train[start:end]
                # # Check batch_y
                # print("Value of batch_y:", batch_y)

                optimizer.zero_grad()
                outputs = self(batch_X)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()
                
                train_loss += loss.item()
                # Calculate accuracy
                num_correct += (self.accuracy_multi(outputs, batch_y, sigmoid=sigmoid) * batch_X.size(0)).item()
                num_samples += batch_X.size(0)
            train_accuracy = (num_correct / num_samples) * 100
            train_accuracies.append(train_accuracy)

            # Validation loss calculation
            self.eval()
            num_val_batches = len(X_val) // batch_size
            num_correct = 0
            num_samples = 0

            for batch in range(num_val_batches):
                start = batch * batch_size
                end = (batch + 1) * batch_size
                batch_X_val = X_val[start:end]
                batch_y_val = y_val[start:end]

                with torch.no_grad():
                    outputs_val = self(batch_X_val) 
                    loss_val = criterion(outputs_val, batch_y_val)
                    val_loss += loss_val.item()
                # #Calculate accuracy
                num_correct += (self.accuracy_multi(outputs_val, batch_y_val, sigmoid=sigmoid) * batch_X_val.size(0)).item()
                num_samples += batch_X_val.size(0)
            # Calculate accuracy for validation at the end of the epoch
            val_accuracy = (num_correct / num_samples) * 100
            val_accuracies.append(val_accuracy)

            train_loss /= num_batches
            val_loss /= num_val_batches
            # scheduler.step(val_loss)  # Update learning rate

            # Append the losses to the lists
            train_losses.append(train_loss)
            val_losses.append(val_loss)

            print(f'Epoch [{epoch+1}/{epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')
            print(f'Epoch [{epoch+1}/{epochs}], Train Acc: {train_accuracy:.4f}, Val Acc: {val_accuracy:.4f}')

            if model_save_path:
                self.save_model(model_save_path)

        # Plot the training and validation loss curves
        plt.figure(figsize=(10, 5))
        plt.plot(range(1, epochs + 1), train_losses, label='Training Loss', marker='o')
        plt.plot(range(1, epochs + 1), val_losses, label='Validation Loss', marker='o')
        plt.title('Training and Validation Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.grid(True)

        # Save the plot as an image
        loss_plot_path = "/global/D1/homes/jayao/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.2/output/custom_test/models/custom/loss_plot.png"
        plt.savefig(loss_plot_path)

    def fine_tune(self, X_train, y_train, X_val, y_val, pretrainedfolder, mpath, sigmoid=True, model_save_path=None):
        # Load the pre-trained model
        pretrained_model = self.load_model(pretrainedfolder, categories=self.num_classes)

        # Set the path for storing the fine-tuned model
        self.outputfolder = mpath

        # Copy the state_dict from the pre-trained model to the current model
        self.load_state_dict(pretrained_model.state_dict())

        # Define your fine-tuning process
        optimizer = optim.Adam(self.parameters(), lr=0.001)
        criterion = nn.BCEWithLogitsLoss()

        # Create a list of layer groups for gradual unfreezing
        layer_groups = [self.conv1, self.block1, self.block2, self.fc1, self.fc2]

        # Discriminative learning rates (adjust as needed)
        lrs = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1]  # Example learning rates

        # Gradual unfreezing and training loop
        for idx, layer_group in enumerate(layer_groups):
            if idx > 0:
                # Unfreeze the next layer group
                for param in layer_group.parameters():
                    param.requires_grad = True

            # Create a new optimizer with the appropriate learning rate
            optimizer = optim.Adam(filter(lambda p: p.requires_grad, self.parameters()), lr=lrs[idx])

            epochs = 10  # Adjust the number of epochs as needed

            # Training loop for the current layer group
            for epoch in range(epochs):
                self.train()
                # Implement your training loop here with the current optimizer
                # Use criterion for loss calculation and optimizer.step() for parameter updates

            # Save the model at the end of each layer group if needed
            if model_save_path:
                self.save_model(model_save_path)

# Create an instance of YourModel
model = YourModel(categories=5)

# Fine-tune the model with discriminative learning rates and gradual unfreezing
model.fine_tune(X_train, y_train, X_val, y_val, pretrainedfolder, mpath)